In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [3]:
filepath = "data_description.txt"

with open(filepath,'r') as file:
    content = file.read()
# print(content) - data description

In [4]:
train = pd.read_csv('train.csv')
print(train.shape)
train.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
test = pd.read_csv('test.csv')
print(test.shape)
test.head()

(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
# concating test and train for EDA
df = pd.concat([train,test],ignore_index=True)

In [7]:
print(train.shape)
print(test.shape)
print(df.shape)

(1460, 81)
(1459, 80)
(2919, 81)


In [8]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.width",None)

## Null treatment

In [9]:
# Test set doesnt have saleprice col, since we concated train and test sale price would be null, so imputed with zero
df.loc[1460:,'SalePrice'] = int(0)

In [10]:
# filtering columns with null values greater than 20% 
null= df.isnull().sum()
null_20perc = null[(null/df.shape[0] * 100)>20]/df.shape[0]
print(null_20perc) 
cols_to_drop = null_20perc.index.to_list()
cols_to_drop

Alley          0.932169
MasVnrType     0.605002
FireplaceQu    0.486468
PoolQC         0.996574
Fence          0.804385
MiscFeature    0.964029
dtype: float64


['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [11]:
# Drop columns with more than 20% null values
print(df.shape, " before dropping")
df = df.drop(columns=cols_to_drop,axis=1)
print(df.shape, " after dropping")

(2919, 81)  before dropping
(2919, 75)  after dropping


In [12]:
others = null[(null.values>0) & (~null.index.isin(null_20perc.index))]
other_null_cols = others.index.to_list()
print(other_null_cols)

['MSZoning', 'LotFrontage', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'SaleType']


In [13]:
# Impute with median and mode for other null columns
for col in other_null_cols:
    if np.issubdtype(df[col].dtype,np.number):
        df[col].fillna(df[col].median(),inplace=True)
    else:
        df[col].fillna(df[col].mode()[0],inplace=True)

In [19]:
null[null>0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType      1766
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

## Univariate Analysis

In [ ]:
num_cols = df.select_dtypes(include=np.number).columns
num_cols

In [ ]:
cat_cols=df.select_dtypes(exclude=np.number).columns
cat_cols

In [ ]:
n_cols = 4  # Number of columns in subplot layout
n_rows = (len(num_cols) + n_cols - 1) // n_cols  # Calculate rows needed

plt.figure(figsize=(12, n_rows * 2))  # Adjust figure size as needed

for k, i in enumerate(num_cols[1:]): #num_cols from 1st index to ignore 'id' column as it is identifier
    plt.subplot(n_rows, n_cols, k + 1)  # Create subplots
    df[i].plot(kind='kde')
    plt.title(i)

plt.tight_layout()  
plt.show() 

In [ ]:
# All the numerical variable is not normally distributed, so we can proceed with tree / ensemble models

In [ ]:
# Outlier Detection using Box Plot
n_cols = 4  
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 2))

for k, i in enumerate(num_cols[1:]): #num_cols from 1st index to ignore 'id'  column as it is identifier
    plt.subplot(n_rows, n_cols, k + 1) 
    df[i].plot(kind='box')
    plt.title(i)

plt.tight_layout()  
plt.show() 

In [ ]:
# There are extreme outliers in almost all features so removing or 
# treating them would affect the data reliability.
# Also we have only 2900+ records, so we can conisder proceeding with the outliers.

## Bivariate Analysis

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[0:4]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[4:8]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[8:12]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[12:16]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[16:20]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[20:24]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[24:28]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[28:32]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
n_cols = 2
n_rows = (len(num_cols) + n_cols - 1) // n_cols  

plt.figure(figsize=(12, n_rows * 3))  
for k,col in enumerate(cat_cols[32:]):
    plt.subplot(n_rows, n_cols, k+1)
    sns.barplot(x=col, y='SalePrice',data=df,estimator=np.mean)
    plt.xticks(rotation=65)
    plt.title(f"SalePrice vs {col}")
    
plt.tight_layout()  
plt.show() 

In [ ]:
corr=df[num_cols].corr()
filtered_corr = corr[(abs(corr)>0.65) & (corr<1)]
sns.heatmap(data=filtered_corr,annot=False, cmap = "RdBu")

In [ ]:
correlated_pairs = filtered_corr.stack().reset_index()
correlated_pairs.columns = ['Column 1', 'Column 2', 'Correlation']
correlated_pairs

## Encoding


In [ ]:
import category_encoders as ce

In [ ]:
freq_code = []
dummy_code = []
for col in cat_cols:
    length=len(df[col].value_counts().index)
    if length>4:
        freq_code.append(col)
    else:
        dummy_code.append(col)

In [ ]:
dummy = ce.OneHotEncoder(cols=dummy_code)
freq = ce.CountEncoder(cols=freq_code)
targ = ce.TargetEncoder()
df_new = df.copy()
df_new = df_new.drop(columns=cat_cols,axis=1)
df_dummy = dummy.fit_transform(df[['Street','LotShape','LandContour','Utilities',
                                   'LandSlope','ExterQual','BsmtQual','BsmtCond',
                                   'BsmtExposure','CentralAir','KitchenQual','GarageFinish',
                                   'PavedDrive']])
df_new = pd.concat([df_new,df_dummy],axis=1)

df_dummy = freq.fit_transform(df[['MSZoning','LotConfig','Neighborhood','Condition1','Condition2',
                                  'BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
                                  'Exterior2nd','ExterCond','Foundation','BsmtFinType1','BsmtFinType2',
                                  'Heating','HeatingQC','Electrical','Functional','GarageType','GarageQual',
                                  'GarageCond','SaleType','SaleCondition']])
df_new = pd.concat([df_new,df_dummy],axis=1)
df_new.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train_df = df_new.iloc[0:1460]
test_df = df_new.iloc[1460:]
test_df = test_df.drop(columns='SalePrice',axis=1)

In [ ]:
test_df.reset_index(inplace=True,drop=True)

In [ ]:
print(train_df.shape)
print(test_df.shape)

## Model Building

In [ ]:
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV, GridSearchCV
from scipy.stats import randint,uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
from xgboost import XGBRegressor 


In [ ]:
xtrain = train_df.drop(columns=['Id','SalePrice'],axis=1)
ytrain = train_df['SalePrice']

In [ ]:
xgb = XGBRegressor(objective='reg:squarederror', n_estimators=100)

xgb.fit(xtrain,ytrain)

In [ ]:
ypred=cross_val_predict(xgb,xtrain,ytrain,cv=5)
print('cross-validation-rmse of XGB',np.sqrt(mse(ytrain,ypred)))

In [ ]:
xgb = XGBRegressor()

xgb_param_grid = {
    'n_estimators':[100,200],
    'max_depth':randint(3,10),
    'learning_rate':uniform(0.01,0.3),
    'subsample': uniform(0.7,0.9)
}

xgb_search = RandomizedSearchCV(estimator=xgb,param_distributions=xgb_param_grid,
                                scoring='neg_mean_squared_error',cv=3,
                               n_iter=50,random_state=11,verbose=1)

xgb_search.fit(xtrain,ytrain)

In [ ]:
best_xgb_model = xgb_search.best_estimator_
best_ypred=cross_val_predict(best_xgb_model,xtrain,ytrain,cv=5)
print('cross-validation-rmse of Tuned XGB ',np.sqrt(mse(ytrain,best_ypred)))

In [ ]:
# GridSearchCV
xgb_params = {'n_estimators': [100, 200],
    'max_depth': [3, 5, 7,10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8,0.9]}
grid_search = GridSearchCV(estimator=xgb, param_grid=xgb_params, 
                           scoring='neg_mean_squared_error', 
                           cv=5, verbose=1, n_jobs=-1)

# Fit the model
grid_search.fit(xtrain, ytrain)

In [ ]:
xgb_gridcv = grid_search.best_estimator_
print("Best Score:", grid_search.best_score_)


In [ ]:
model = XGBRegressor()

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'n_estimators': [100, 200]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5)
grid_search.fit(xtrain, ytrain)

In [ ]:
xgb_2 = grid_search.best_estimator_
print("Best Score:",grid_search.best_score_)

In [ ]:
test_df.head()

In [ ]:
preds = xgb_gridcv.predict(test_df.drop(columns='Id',axis=1))
preds

In [ ]:
submission = pd.concat([test_df['Id'],pd.Series(preds)],axis=1,ignore_index=True)
submission.columns = ['Id','SalePrice']
submission.head()

In [ ]:
submission1.to_csv('submission.csv',index=False)